In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from config import api_key
from scipy.stats import linregress
from pprint import pprint

In [ ]:
artist = 'FooFighters'
locale = 'en-us'
segment = 'Music'
events = requests.get(f'https://app.ticketmaster.com/discovery/v2/events.json?keyword={artist}&segment{segment}&locale={locale}&apikey={api_key}').json()

pprint(events)

{'_embedded': {'events': [{'_embedded': {'attractions': [{'_links': {'self': {'href': '/discovery/v2/attractions/K8vZ91713Y7?locale=en-us'}},
                                                          'aliases': ['foo '
                                                                      'fighters',
                                                                      'fu '
                                                                      'fighters'],
                                                          'classifications': [{'family': False,
                                                                               'genre': {'id': 'KnvZfZ7vAeA',
                                                                                         'name': 'Rock'},
                                                                               'primary': True,
                                                                               'segment': {'id': 'KZFzniwnSyZfZ7v7nJ',
             